In [43]:
%load_ext autoreload
%autoreload 2
from mocca.dad_data.apis.chemstation_api import read_csv_agilent, tidy_df_agilent
from mocca.dad_data.apis.labsolutions_api import read_txt_shimadzu
import os
from mocca.peak.models import ProcessedPeak
from mocca.peak.database import PeakDatabase
from mocca.components.utils import get_filtered_peaks_by_compound

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df = read_csv_agilent(r"/Users/haascp/Documents/GitHub/mocca/tests/example_data/chemstation_api/test_data_chemstation")

In [7]:
len(df)

6603

In [8]:
len(df.columns)

347

In [9]:
df

,Unnamed: 0,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,...,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550
0,0.000617,2.6,2.8,2.5,2.0,1.8,1.7,1.5,1.5,1.5,1.3,1.0,0.5,0.6,0.6,0.4,0.3,0.3,0.2,0.0,0.1,-0.1,-0.0,-0.1,-0.0,-0.2,-0.3,-0.5,-0.2,-0.4,-0.4,-0.4,-0.3,-0.5,-0.5,-0.4,-0.4,-0.5,-0.4,-0.4,...,-0.5,-0.5,-0.4,-0.7,-0.6,-0.3,-0.4,-0.5,-0.7,-0.6,-0.5,-0.8,-0.3,-0.5,-0.6,-0.2,-0.3,-0.4,-0.6,-0.3,-0.5,-0.6,-0.1,-0.5,-0.5,-0.7,-0.4,-0.7,-0.4,-0.2,-0.5,-0.6,-0.8,-0.3,-0.6,-0.4,-0.3,-0.4,-0.8,-0.7
1,0.001450,2.6,2.7,2.4,2.0,1.8,1.7,1.5,1.5,1.5,1.3,0.9,0.5,0.6,0.5,0.4,0.3,0.3,0.2,0.0,0.0,-0.1,-0.0,-0.1,-0.1,-0.3,-0.4,-0.5,-0.2,-0.4,-0.4,-0.4,-0.3,-0.5,-0.5,-0.4,-0.4,-0.5,-0.4,-0.4,...,-0.5,-0.4,-0.5,-0.7,-0.6,-0.4,-0.4,-0.5,-0.7,-0.6,-0.5,-0.8,-0.3,-0.6,-0.6,-0.1,-0.4,-0.4,-0.7,-0.3,-0.5,-0.6,-0.1,-0.5,-0.6,-0.7,-0.4,-0.7,-0.4,-0.2,-0.5,-0.7,-0.8,-0.3,-0.6,-0.4,-0.3,-0.4,-0.9,-0.7
2,0.002283,2.6,2.7,2.4,2.1,1.8,1.7,1.5,1.4,1.4,1.3,0.9,0.5,0.6,0.5,0.4,0.3,0.2,0.2,0.0,0.0,-0.1,-0.0,-0.1,-0.1,-0.3,-0.4,-0.5,-0.2,-0.4,-0.4,-0.4,-0.4,-0.5,-0.5,-0.4,-0.5,-0.5,-0.4,-0.4,...,-0.6,-0.4,-0.5,-0.7,-0.6,-0.4,-0.4,-0.4,-0.7,-0.6,-0.5,-0.8,-0.3,-0.6,-0.6,-0.1,-0.4,-0.4,-0.7,-0.3,-0.6,-0.5,-0.2,-0.5,-0.6,-0.7,-0.4,-0.6,-0.4,-0.3,-0.5,-0.7,-0.8,-0.4,-0.6,-0.4,-0.4,-0.4,-1.0,-0.7
3,0.003117,2.6,2.7,2.4,2.1,1.8,1.7,1.5,1.4,1.4,1.2,0.9,0.5,0.6,0.5,0.4,0.3,0.2,0.2,0.0,0.0,-0.1,-0.0,-0.2,-0.1,-0.3,-0.4,-0.5,-0.2,-0.4,-0.4,-0.4,-0.4,-0.5,-0.5,-0.4,-0.5,-0.5,-0.4,-0.4,...,-0.6,-0.4,-0.5,-0.7,-0.6,-0.5,-0.5,-0.4,-0.7,-0.7,-0.5,-0.8,-0.2,-0.7,-0.6,-0.2,-0.4,-0.4,-0.7,-0.4,-0.6,-0.5,-0.3,-0.5,-0.7,-0.7,-0.4,-0.6,-0.5,-0.4,-0.5,-0.7,-0.8,-0.5,-0.6,-0.4,-0.4,-0.5,-1.0,-0.7
4,0.003950,2.7,2.7,2.4,2.2,1.9,1.8,1.5,1.4,1.3,1.2,1.0,0.6,0.7,0.5,0.4,0.3,0.3,0.2,0.1,0.0,-0.1,-0.0,-0.2,-0.1,-0.3,-0.3,-0.4,-0.3,-0.4,-0.4,-0.4,-0.4,-0.5,-0.5,-0.4,-0.5,-0.5,-0.4,-0.4,...,-0.6,-0.4,-0.5,-0.7,-0.6,-0.6,-0.5,-0.4,-0.7,-0.7,-0.5,-0.7,-0.2,-0.7,-0.6,-0.3,-0.4,-0.4,-0.7,-0.4,-0.6,-0.5,-0.3,-0.5,-0.7,-0.6,-0.4,-0.6,-0.5,-0.5,-0.5,-0.7,-0.8,-0.5,-0.6,-0.4,-0.4,-0.6,-1.0,-0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6598,5.498950,-95.0,-81.3,-66.3,-50.6,-35.4,-20.0,-6.1,7.3,18.6,28.9,37.0,43.9,49.3,53.3,56.0,57.7,58.7,59.0,58.7,57.9,57.0,55.9,54.4,52.8,51.0,49.0,46.9,44.7,42.4,40.2,37.8,35.5,33.1,30.7,28.6,26.2,24.1,22.1,20.1,...,0.6,0.5,0.6,0.6,0.4,0.3,0.5,0.8,0.4,0.4,0.3,0.6,0.7,0.3,0.5,0.4,0.6,0.5,0.3,0.5,0.4,0.5,0.6,0.4,0.2,0.3,0.5,0.3,0.7,0.5,0.5,0.3,0.3,0.5,0.3,0.5,0.5,0.4,0.1,0.4
6599,5.499783,-95.2,-81.5,-66.4,-50.7,-35.5,-20.1,-6.1,7.2,18.5,28.8,36.9,44.0,49.3,53.3,56.1,57.7,58.8,59.0,58.7,58.0,57.0,55.9,54.4,52.8,51.0,49.1,46.9,44.7,42.4,40.3,37.8,35.5,33.2,30.8,28.7,26.3,24.1,22.1,20.1,...,0.6,0.4,0.6,0.6,0.4,0.3,0.5,0.7,0.4,0.4,0.4,0.6,0.7,0.3,0.5,0.4,0.6,0.5,0.3,0.6,0.3,0.5,0.6,0.4,0.3,0.3,0.5,0.3,0.7,0.5,0.5,0.4,0.3,0.5,0.4,0.5,0.5,0.5,0.3,0.4
6600,5.500617,-95.4,-81.6,-66.5,-50.8,-35.6,-20.2,-6.2,7.2,18.5,28.7,36.9,44.0,49.2,53.3,56.1,57.8,58.8,59.0,58.7,58.0,57.1,55.9,54.5,52.8,51.1,49.1,46.9,44.7,42.5,40.3,37.8,35.5,33.2,30.8,28.7,26.3,24.1,22.1,20.1,...,0.7,0.4,0.5,0.6,0.5,0.4,0.5,0.7,0.4,0.5,0.4,0.5,0.7,0.3,0.5,0.4,0.6,0.5,0.4,0.6,0.4,0.5,0.5,0.5,0.4,0.3,0.5,0.3,0.7,0.5,0.5,0.4,0.4,0.5,0.4,0.4,0.5,0.5,0.4,0.4
6601,5.501450,-95.6,-81.8,-66.6,-50.9,-35.7,-20.2,-6.2,7.2,18.4,28.7,36.9,44.0,49.2,53.3,56.1,57.8,58.8,59.1,58.8,58.1,57.1,55.8,54.5,52.9,51.1,49.1,47.0,44.8,42.5,40.3,37.9,35.5,33.2,30.8,28.7,26.3,24.1,22.2,20.1,...,0.7,0.4,0.4,0.6,0.5,0.5,0.5,0.6,0.3,0.

In [15]:
tidy_df_agilent(df)

,time,wavelength,absorbance
0,0.000833,205.0,2.6
1,0.001667,205.0,2.6
2,0.002500,205.0,2.6
3,0.003333,205.0,2.6
4,0.004167,205.0,2.7
...,...,...,...
2284633,5.498950,550.0,0.4
2284634,5.499783,550.0,0.4
2284635,5.500616,550.0,0.4
2284636,5.501450,550.0,0.4


In [19]:
df = read_txt_shimadzu(r"/Users/haascp/Documents/GitHub/mocca/tests/example_data/labsolutions_api/labsolutions_test_data.txt")

In [21]:
df.wavelength.nunique()

164

In [25]:
max_conc_dict = {'a': 1, 'b': 2}

In [26]:
for key, val in max_conc_dict.items():
    print(key)
    print(val)

a
1
b
2


In [24]:
any(val == 5 for val in x.values())

True

In [27]:
def add_numbers(x, y):
    z = x + y
    return z

In [28]:
add_numbers(1,4)

5

In [37]:
peaks = [ProcessedPeak(left=87, right=113, maximum=100, dataset="1", idx=0, saturation=False, pure=True, integral=2.2191571128391883, offset=0, compound_id='1', concentration=2.2191571128391883), 
         ProcessedPeak(left=125, right=175, maximum=150, dataset="1", idx=1, saturation=False, pure=True, integral=3.125566226400256, offset=0, compound_id='2', concentration=1.562783113200128), 
         ProcessedPeak(left=186, right=213, maximum=200, dataset="1", idx=2, saturation=False, pure=True, integral=2.223456694964788, offset=0, compound_id='3', concentration=0.7411522316549294), 
         ProcessedPeak(left=236, right=263, maximum=250, dataset="1", idx=3, saturation=False, pure=True, integral=2.222925062035727, offset=0, compound_id='4', concentration=0.5557312655089317), 
         ProcessedPeak(left=291, right=308, maximum=300, dataset="1", idx=4, saturation=False, pure=True, integral=1.7261150928159654, offset=0, compound_id='5', concentration=0.3452230185631931), 
         ProcessedPeak(left=375, right=425, maximum=400, dataset="1", idx=5, saturation=False, pure=True, integral=3.12480200703372, offset=0, compound_id='6', concentration=0.52080033450562), 
         ProcessedPeak(left=491, right=508, maximum=500, dataset="1", idx=6, saturation=False, pure=True, integral=1.7259476202140827, offset=0, compound_id='7', concentration=0.24656394574486895), 
         ProcessedPeak(left=684, right=715, maximum=699, dataset="1", idx=7, saturation=False, pure=True, integral=2.4285252194452736, offset=0, compound_id='8', concentration=0.3035656524306592), 
         ProcessedPeak(left=763, right=824, maximum=799, dataset="1", idx=8, saturation=False, pure=False, integral=3.8583503915259927, offset=0, compound_id='9', concentration=0.42870559905844363), 
         ProcessedPeak(left=824, right=886, maximum=849, dataset="1", idx=9, saturation=False, pure=False, integral=3.88012983036313, offset=0, compound_id='10', concentration=0.388012983036313)]

In [40]:
peak_db = PeakDatabase()
peak_db.peaks = peaks

In [44]:
get_filtered_peaks_by_compound(peak_db, None)

{'1': [ProcessedPeak(left=87, right=113, maximum=100, dataset='1', idx=0, saturation=False, pure=True, integral=2.2191571128391883, offset=0, compound_id='1', concentration=2.2191571128391883)],
 '2': [ProcessedPeak(left=125, right=175, maximum=150, dataset='1', idx=1, saturation=False, pure=True, integral=3.125566226400256, offset=0, compound_id='2', concentration=1.562783113200128)],
 '3': [ProcessedPeak(left=186, right=213, maximum=200, dataset='1', idx=2, saturation=False, pure=True, integral=2.223456694964788, offset=0, compound_id='3', concentration=0.7411522316549294)],
 '4': [ProcessedPeak(left=236, right=263, maximum=250, dataset='1', idx=3, saturation=False, pure=True, integral=2.222925062035727, offset=0, compound_id='4', concentration=0.5557312655089317)],
 '5': [ProcessedPeak(left=291, right=308, maximum=300, dataset='1', idx=4, saturation=False, pure=True, integral=1.7261150928159654, offset=0, compound_id='5', concentration=0.3452230185631931)],
 '6': [ProcessedPeak(left

In [47]:
x = False

In [5]:
class A():
    def __init__(self, test):
        self.test = test
    

In [8]:
test_obj = A(False)

In [9]:
test_obj.test

False

In [11]:
if test_obj.test is False:
    print('x')

x


In [12]:
len_counter = 1

In [13]:
f"hello {len_counter} yo"

'hello 1 yo'

In [14]:
x = [1]

In [15]:
x.pop(0)

1

In [16]:
x

[]